In [1]:
#import os
#os.chdir('../')
#os.getcwd()

'/Users/amuhebwa/Documents/Python/Predicting_River_Discharge/Create_Dataset'

In [2]:
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
ee.Initialize()

In [4]:
# 1. Load shapefile with all polygons
upstream_data = gpd.read_file('GEE_makenzie_basins_climate/GEE_makenzie_basins_climate.shp')

In [5]:
upstream_data.head()

,Stn_COMID,Stn_Name,COMID,lat,lon,PIXEL_ID,PERC_AREA,geometry
0,82013426,07SB019,82013427,64.375,-113.375,1331,0.19,"POLYGON ((-113.35042 64.25000, -113.35042 64.2..."
1,82013426,07SB019,82013427,64.375,-113.125,1332,2.09,"POLYGON ((-113.25000 64.28958, -113.24958 64.2..."
2,82013426,07SB019,82013427,64.375,-112.875,1333,2.16,"POLYGON ((-113.00000 64.32458, -112.99958 64.3..."
3,82013426,07SB019,82013427,64.375,-112.625,1334,0.12,"POLYGON ((-112.75000 64.31958, -112.74375 64.3..."
4,82013426,07SB019,82013427,64.125,-113.625,1434,0.10,"MULTIPOLYGON (((-113.57042 64.00000, -113.5704..."


In [6]:
# 2. Filter out upstream polgons for one guage station
upstream_data = upstream_data[upstream_data['Stn_Name']=='07AA002'].reset_index(drop=True)

In [7]:
upstream_data

,Stn_COMID,Stn_Name,COMID,lat,lon,PIXEL_ID,PERC_AREA,geometry
0,82042777,07AA002,82042973,53.125,-118.625,5099,1.32,"POLYGON ((-118.64542 53.00000, -118.64542 53.0..."
1,82042777,07AA002,82042973,53.125,-118.375,5100,0.43,"POLYGON ((-118.50000 53.05708, -118.49625 53.0..."
2,82042777,07AA002,82042973,52.875,-118.625,5114,2.84,"POLYGON ((-118.50000 52.90542, -118.50125 52.9..."
3,82042777,07AA002,82042973,52.875,-118.375,5115,8.40,"POLYGON ((-118.35542 52.75000, -118.35542 52.7..."
4,82042777,07AA002,82042973,52.875,-118.125,5116,6.80,"POLYGON ((-118.25000 52.93292, -118.24958 52.9..."
5,82042777,07AA002,82042973,52.875,-117.875,5117,2.43,"POLYGON ((-118.00000 52.85708, -117.99958 52.8..."
6,82042777,07AA002,82042973,52.625,-118.375,5126,3.09,"POLYGON ((-118.25292 52.50000, -118.25292 52.5..."
7,82042777,07AA002,82042973,52.625,-118.125,5127,12.33,"POLYGON ((-118.00000 52.50000, -118.25000 52.5..."
8,82042777,07AA002,82042973,52.625,-117.875,5128,11.96,"POLYGON ((-117.82792 52.75000, -117.82792 52.7..."
9,82042777,07AA002,82042973,52.625,-117.625,5129,4.24,"MULTIPOLYGON (((-117.75000 52.66042, -117.7495..."


In [ ]:
#3. Select image collection for climate data
climate_data = ee.ImageCollection('NASA/GLDAS/V20/NOAH/G025/T3H')

In [ ]:
#4. select only one band, for Jan-02-1981
start_date = '1981-01-02'
end_date = '1981-01-03'
bands_to_select = ["RootMoist_inst"]
climate_data = climate_data.select(bands_to_select).filterDate(start_date, end_date)

In [ ]:
"""
convert earth engine array to a dataframe
There is no data manipulation here, except changing the data structure
from an array to dataframe
"""
def array_to_df(arr:np.array,pixelId:np.int64, pixelWeight:np.float64, bands:np.array) -> pd.DataFrame:
    df = pd.DataFrame(arr)
    column_names = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=column_names)
    df = df[['longitude', 'latitude', 'time', *bands]].dropna()
    df[bands] = df[bands].astype(np.float)
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    df = df[['datetime','longitude', 'latitude', *bands]]
    df['PIXEL_ID'] = pixelId
    return df

In [ ]:
'''
5. 
We could loop over the upstream data frame and extract the data from each row
but it is slow.
Best approach: convert columns to arrays and zip them
Also, define the projection, which is each to the 0.25 arc degrees 
'''
lats = upstream_data.lat.values
lons = upstream_data.lon.values
pixel_ids = upstream_data.PIXEL_ID.values
percent_areas = upstream_data.PERC_AREA.values
pixel_scale = climate_data.first().projection().nominalScale()

In [ ]:
df_arrs = []
for lon, lat, pixel_id, perc_area in zip(lons, lats, pixel_ids, percent_areas):
    point = {'type': 'Point', 'coordinates': (lon, lat)} # convert lat/lon to GEE point
    current_aoi = climate_data.getRegion(point, pixel_scale).getInfo() # extract climate data for that pixel
    ## convert the extracted array to dataframe. There is no data manipulation except changing the data format
    pixel_df = array_to_df(current_aoi,pixel_id, perc_area, bands_to_select)
    df_arrs.append(pixel_df)
    print('Pixel ID : {} Completed'.format(pixel_id))

In [ ]:
final_df = pd.concat(df_arrs, axis=0).reset_index(drop=True) # join all pixel dataframes to a single dataframe

In [ ]:
final_df